In [ ]:
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)
# import climakitae as ck

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.backends.backend_pdf import PdfPages
import time
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx

print('done')

In [ ]:
variables_units = ("Fosberg fire weather index","[0 to 100]","Dynamical","hourly",[2.0])
variable, unit,downscale,timescale,GWL = variables_units
print(variable,unit,downscale,timescale,GWL)
print(f"Processing variable: {variable}")
ds = get_data(
    variable=variable,
    # units=unit,
    downscaling_method=downscale,
    resolution="3 km",
    timescale=timescale,
    cached_area="Southern California Edison",
    approach="Warming Level",
    warming_level_window=15,
    warming_level=GWL
)

hours_in_year = 365 * 24
# Calculate the hour of the year
hour_of_year = ds['time_delta'].values % hours_in_year
# Assign the new hour_of_year coordinate to the dataset
ds = ds.assign_coords(hour_of_year=('time_delta', hour_of_year))
print(ds)
ds = ds.drop_vars(['landmask', 'lakemask'])
hourly_mean = ds.groupby('hour_of_year').mean(dim='time_delta', skipna=True)
print(hourly_mean)
del ds
hours_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) * 24
cumulative_hours = np.cumsum(hours_in_month)
# Function to map hour_of_year to month
def hour_to_month(hour):
    return np.searchsorted(cumulative_hours, hour) + 1

Apply the function to calculate the month for each hour_of_year

print(hourly_mean)
months = hour_to_month(hourly_mean['hour_of_year'].values)
hourly_mean = hourly_mean.assign_coords(month=('hour_of_year', months))

monthly_max = hourly_mean.groupby('month').max(dim='hour_of_year', skipna=True)
print(monthly_max)

In [ ]:
percentiles = [10,50,90]
percentile_values_ts = {p: hourly_mean.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}

In [ ]:
monthly_max.to_netcdf('Fosberg fire weather index_GWL2.0_MonthlyMax.nc')

In [3]:
del ds
hourly_mean = hourly_mean.drop_vars(['landmask', 'lakemask'])
percentiles = [10,50,90]
percentile_values_ts = {p: hourly_mean.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}
print(percentile_values_ts)
# hourly_mean.to_netcdf('Fosberg fire weather index_GWL2.0_8760_climo.nc')

{10: <xarray.DataArray np.str_('Fosberg fire weather index') (warming_level: 1,
                                                         hour_of_year: 8760,
                                                         y: 240, x: 149)> Size: 3GB
dask.array<getitem, shape=(1, 8760, 240, 149), dtype=float64, chunksize=(1, 117, 240, 149), chunktype=numpy.ndarray>
Coordinates:
  * warming_level  (warming_level) float64 8B 2.0
  * x              (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y              (y) float64 2kB 5.899e+05 5.929e+05 ... 1.304e+06 1.307e+06
    lat            (y, x) float32 143kB 31.49 31.5 31.52 ... 39.18 39.19 39.21
    lon            (y, x) float32 143kB -117.7 -117.7 -117.7 ... -118.2 -118.2
  * hour_of_year   (hour_of_year) float64 70kB 0.0 1.0 ... 8.758e+03 8.759e+03
    quantile       float64 8B 0.1
Attributes:
    variable_id:           fosberg_index_derived
    extended_description:  Fire weather index using meteorological inputs (te...
    units:     

In [ ]:
percentile_values_ts[10].to_netcdf('Fosberg fire weather index_GWL2.0_8760_climo_GWL2.0_10th.nc')
# percentile_values_ts[50].to_netcdf('Fosberg fire weather index_GWL2.0_8760_climo_GWL2.0_50th.nc')
# percentile_values_ts[90].to_netcdf('Fosberg fire weather index_GWL2.0_8760_climo_GWL2.0_90th.nc')

In [5]:
percentiles = [10,50,90]
percentile_values_ts = {p: hourly_mean.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}
percentile_values = {p: monthly_max.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}

In [2]:
variables_units = ("Precipitation (total)", "inches","Statistical","daily",[1.0,1.5,2.0])
variable, unit,downscale,timescale,GWL = variables_units
print(variable,unit,downscale,timescale,GWL)
print(f"Processing variable: {variable}")
ds = get_data(
    variable=variable,
    units=unit,
    downscaling_method=downscale,
    resolution="3 km",
    timescale=timescale,
    cached_area="Southern California Edison",
    approach="Warming Level",
    warming_level_window=15,
    warming_level=GWL
)
print(ds)

Precipitation (total) inches Statistical daily [1.0, 1.5, 2.0]
Processing variable: Precipitation (total)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

WARNING FOR WARMING LEVELS APPROACH
-----------------------------------
There may be NaNs in your data for certain simulation/warming level combinations if the warming level is not reached for that particular simulation before the year 2100. 

This does not mean you have missing data, but rather a feature of how the data is combined in retrieval to return a single data object. 

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual warming level and then dropping NaN values.
<xarray.DataArray np.str_('Precipitation (total)') (wa